In [1]:
# Author: Dýrmundur Helgi
# Date: 24.8.2023
# Project: Assignment 1 - Decision Trees
# Acknowledgements: 
#


from typing import Union
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier, plot_tree

from tools import load_iris, split_train_test

In [2]:
# Part 1.1
def prior(targets: np.ndarray, classes: list) -> np.ndarray:
    '''
    Calculate the prior probability of each class type
    given a list of all targets and all class types
    '''
    probabilities = np.zeros(len(classes))
    
    for i, a_class in enumerate(classes):
        for target in targets:
            if target == a_class:
                probabilities[i] += 1
    return probabilities / len(targets)

print(prior([0, 0, 1], [0, 1]))
print(prior([0, 2, 3, 3], [0, 1, 2, 3]))


[0.66666667 0.33333333]
[0.25 0.   0.25 0.5 ]


In [17]:
# Part 1.2
def split_data(
    features: np.ndarray,
    targets: np.ndarray,
    split_feature_index: int,
    theta: float
) -> Union[tuple, tuple]:
    '''
    Split a dataset and targets into two seperate datasets
    where data with split_feature < theta goes to 1 otherwise 2
    '''
    condition = features[:, split_feature_index] < theta
    
    features_1 = features[condition]
    targets_1 = targets[condition]

    features_2 = features[~condition]
    targets_2 = targets[~condition]

    return (features_1, targets_1), (features_2, targets_2)


features, targets, classes = load_iris()
(f_1, t_1), (f_2, t_2) = split_data(features, targets, 2, 4.65)
print(len(f_1), len(f_2))


90 60


In [7]:
def using_iris():
    '''
    Shows how load_iris works
    '''
    # 1. Load the Iris dataset:
    features, targets, classes = load_iris()

    [n, f_dim] = features.shape

    print(f'* The dataset contains {n} samples')
    print(f'* Each sample has {f_dim} features')

    # 2. get the first datapoint
    first_feature_set = features[0, :]
    first_target = targets[0]

    print(f'* The first sample has the features:\n{first_feature_set} '+\
        f'and belongs to the class {first_target}')

    print('* Each datapoint can belong to any of the following classes:'+\
        f'\n{classes}')

    # 3. Split into train and test sets
    (train_features, train_targets), (test_features, test_targets) =\
        split_train_test(features, targets, train_ratio=0.9)

    train_n, test_n = train_features.shape[0], test_features.shape[0]
    print(f'* The train set contains {train_n} samples')
    print(f'* The test set contains {test_n} samples')

using_iris()

* The dataset contains 150 samples
* Each sample has 4 features
* The first sample has the features:
[5.1 3.5 1.4 0.2] and belongs to the class 0
* Each datapoint can belong to any of the following classes:
[0, 1, 2]
* The train set contains 135 samples
* The test set contains 14 samples


In [19]:
# Part 1.3
def gini_impurity(targets: np.ndarray, classes: list) -> float:
    '''
    Calculate:
        i(S_k) = 1/2 * (1 - sum_i P{C_i}**2)
    '''
    return 0.5 * (1 - np.sum(np.power(prior(targets, classes),2)))

print(gini_impurity(t_1, classes))
print(gini_impurity(t_2, classes))

0.2517283950617284
0.1497222222222222


In [21]:
def weighted_impurity(
    t1: np.ndarray,
    t2: np.ndarray,
    classes: list
) -> float:
    '''
    Given targets of two branches, return the weighted
    sum of gini branch impurities
    '''
    g1 = gini_impurity(t1, classes)
    g2 = gini_impurity(t2, classes)
    n = t1.shape[0] + t2.shape[0]
    
    return (len(t1)*g1)/n + (len(t2)*g2)/n

print(weighted_impurity(t_1, t_2, classes))

0.2109259259259259

In [22]:
# Part 1.5
def total_gini_impurity(
    features: np.ndarray,
    targets: np.ndarray,
    classes: list,
    split_feature_index: int,
    theta: float
) -> float:
    '''
    Calculate the gini impurity for a split on split_feature_index
    for a given dataset of features and targets.
    '''
    (f_1, t_1), (f_2, t_2) = split_data(features, targets, split_feature_index, theta)
    return weighted_impurity(t_1, t_2, classes)

print(total_gini_impurity(features, targets, classes, 2, 4.65))

0.2109259259259259


In [24]:
# Part 1.6
def brute_best_split(
    features: np.ndarray,
    targets: np.ndarray,
    classes: list,
    num_tries: int
) -> Union[float, int, float]:
    '''
    Find the best split for the given data. Test splitting
    on each feature dimension num_tries times.

    Return the lowest gini impurity, the feature dimension and
    the threshold
    '''
    best_gini, best_dim, best_theta = float("inf"), None, None
    # iterate feature dimensions
    for i in range(features.shape[1]):
        # create the thresholds
        row = features[:,i]
        thetas = np.linspace(np.min(row), np.max(row),num_tries+2)[1:-1]
        # iterate thresholds
        for theta in thetas:
            gini = total_gini_impurity(features, targets, classes, i, theta)
            if gini < best_gini:
                best_gini = gini
                best_dim = i
                best_theta = theta
    return best_gini, best_dim, best_theta

print(brute_best_split(features, targets, classes, 30))

(0.16666666666666666, 2, 1.9516129032258065)


In [3]:
class IrisTreeTrainer:
    def __init__(
        self,
        features: np.ndarray,
        targets: np.ndarray,
        classes: list = [0, 1, 2],
        train_ratio: float = 0.8
    ):
        '''
        train_ratio: The ratio of the Iris dataset that will
        be dedicated to training.
        '''
        (self.train_features, self.train_targets),\
            (self.test_features, self.test_targets) =\
            split_train_test(features, targets, train_ratio)

        self.classes = classes
        self.tree = DecisionTreeClassifier()

    def train(self):
        ...

    def accuracy(self):
        ...

    def plot(self):
        ...

    def plot_progress(self):
        # Independent section
        # Remove this method if you don't go for independent section.
        ...

    def guess(self):
        ...

    def confusion_matrix(self):
        ...